In [ ]:
!nvidia-smi

Tue Feb 10 07:59:30 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             12W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import drive
!fusermount -uz /content/drive # Force unmount if previous mount is corrupted
drive.mount('/content/drive')

fusermount: failed to unmount /content/drive: No such file or directory


ValueError: mount failed

In [ ]:
%cd /content/
!git clone https://github.com/godspeed-003/lora-slm-finetuning.git

/content
fatal: destination path 'lora-slm-finetuning' already exists and is not an empty directory.


In [ ]:
%cd lora-slm-finetuning

/content/lora-slm-finetuning


In [ ]:
!pip install -q torch transformers datasets peft accelerate wandb PyYAML tqdm

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4


In [ ]:
import wandb

# Login to W&B
wandb.login()  # This will prompt for your API key

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Create a new API key at: https://wandb.ai/authorize?ref=models
wandb: Store your API key securely and do not share it.
wandb: Paste your API key and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vedantkorade (vedantkorade-mumbai-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Pull latest code from GitHub
%cd /content/lora-slm-finetuning
!git pull origin main

!ls -la src/
!ls -la configs/

/content/lora-slm-finetuning
From https://github.com/godspeed-003/lora-slm-finetuning
 * branch            main       -> FETCH_HEAD
Already up to date.
total 32
drwxr-xr-x 2 root root  4096 Feb 10 07:34 .
drwxr-xr-x 6 root root  4096 Feb 10 07:34 ..
-rw-r--r-- 1 root root  4032 Feb 10 07:34 data_loader.py
-rw-r--r-- 1 root root     0 Feb 10 07:34 inference.py
-rw-r--r-- 1 root root 10049 Feb 10 07:34 train_enhanced.py
-rw-r--r-- 1 root root  4897 Feb 10 07:34 train.py
-rw-r--r-- 1 root root     0 Feb 10 07:34 utils.py
total 16
drwxr-xr-x 2 root root 4096 Feb 10 07:34 .
drwxr-xr-x 6 root root 4096 Feb 10 07:34 ..
-rw-r--r-- 1 root root 1088 Feb 10 07:34 datasets.yaml
-rw-r--r-- 1 root root  896 Feb 10 07:34 training_config.yaml


In [ ]:
import sys
sys.path.append('/content/lora-slm-finetuning/src')

In [ ]:
# Delete cached datasets
!rm -rf ~/.cache/huggingface/datasets/*
!rm -rf /root/.cache/huggingface/datasets/*

# Delete downloaded models (we can re-download)
!rm -rf models/*

# Check disk usage
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay         113G   66G   48G  58% /
tmpfs            64M     0   64M   0% /dev
shm             5.7G     0  5.7G   0% /dev/shm
/dev/root       2.0G  1.2G  750M  62% /usr/sbin/docker-init
tmpfs           6.4G  424K  6.4G   1% /var/colab
/dev/sda1       119G  104G   16G  88% /kaggle/input
tmpfs           6.4G     0  6.4G   0% /proc/acpi
tmpfs           6.4G     0  6.4G   0% /proc/scsi
tmpfs           6.4G     0  6.4G   0% /sys/firmware
drive           113G   68G   46G  60% /content/drive


In [ ]:
# Stop any running cells first!

# Clean up
!rm -rf ~/.cache/huggingface/datasets/*
!pkill -9 python3  # Force kill any hanging processes

# Check disk space
!df -h /

In [ ]:
# In Colab - pull latest
%cd /content/lora-slm-finetuning
!git pull origin main

# Verify datasets config
!cat configs/datasets.yaml

/content/lora-slm-finetuning
From https://github.com/godspeed-003/lora-slm-finetuning
 * branch            main       -> FETCH_HEAD
Already up to date.
# Dataset Configuration for LoRA Fine-tuning
# OPTIMIZED FOR COLAB FREE (3-4 hour training time)

datasets:
  # Core LM Training (3 datasets)
  - name: "wikitext"
    config: "wikitext-103-raw-v1"
    split: "train"
    text_field: "text"

  - name: "roneneldan/TinyStories"
    config: "default"
    split: "train"
    text_field: "text"

  # News (2 datasets)
  - name: "ag_news"
    config: "default"
    split: "train"
    text_field: "text"

  - name: "yelp_review_full"
    config: "default"
    split: "train"
    text_field: "text"

  # Summarization (2 datasets)
  - name: "cnn_dailymail"
    config: "3.0.0"
    split: "train"
    text_field: "article"

  - name: "xsum"
    config: "default"
    split: "train"
    text_field: "document"

  # QA & Sentiment (2 datasets)
  - name: "squad"
    config: "plain_text"
    split: "train"
    

In [ ]:
# Remove all Hugging Face cache (nuclear option but safe)
!rm -rf ~/.cache/huggingface/*
!rm -rf /root/.cache/huggingface/*

# Remove any partial downloads
!find /tmp -name "*.tmp" -delete
!find /tmp -name "*.parquet" -delete

# Check disk space
!df -h / | grep overlay

# Should now show more free space

overlay         113G   39G   74G  35% /


In [ ]:
import sys
sys.path.append('/content/lora-slm-finetuning/src')

# Force reload modules to pick up new config
if 'train_enhanced' in sys.modules:
    del sys.modules['train_enhanced']
if 'data_loader' in sys.modules:
    del sys.modules['data_loader']

from train_enhanced import train_enhanced

# Start training with the fixed config
print("🚀 Starting training with 10 datasets...")
print("=" * 60)
train_enhanced()

🚀 Starting training with 10 datasets...


wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: vedantkorade (vedantkorade-mumbai-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Loading tokenizer...


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475

🚀 Starting training...


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Run this in a separate cell while training runs
import time
import shutil
from pathlib import Path

def backup_checkpoints():
    """Periodically backup checkpoints to Google Drive"""
    while True:
        time.sleep(1800)  # Every 30 minutes

        output_dir = Path('/content/lora-slm-finetuning/outputs')
        if output_dir.exists():
            backup_dir = Path('/content/drive/MyDrive/lora_checkpoints_backup')
            backup_dir.mkdir(parents=True, exist_ok=True)

            # Copy latest checkpoint
            checkpoints = list(output_dir.glob('checkpoint-*'))
            if checkpoints:
                latest = max(checkpoints, key=lambda p: p.stat().st_mtime)
                dest = backup_dir / latest.name
                if not dest.exists():
                    shutil.copytree(latest, dest)
                    print(f"✓ Backed up {latest.name} to Drive")

In [ ]:
import sys
sys.path.append('/content/lora-slm-finetuning/src')

# Clear cache
if 'train_enhanced' in sys.modules:
    del sys.modules['train_enhanced']
if 'data_loader' in sys.modules:
    del sys.modules['data_loader']

from train_enhanced import train_enhanced

print("🚀 Starting OPTIMIZED training...")
train_enhanced()

🚀 Starting OPTIMIZED training...


dataset/eval_samples,▁
dataset/num_datasets,▁
dataset/train_samples,▁
step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train/bpt,▆█▇▆█▇▄▅▅▃▄▄▆▄▃▄▂▃▃▄▃▃▁▅▅▃
train/ema_token_loss,▇▇▇▇███▇▇▆▆▅▆▅▅▄▄▃▃▂▂▂▁▁▁▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▆▆▆▆▇▇▇▇██
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
train/grad_norm,▂▂▂▁▄▃▄▃▃██▆▇▇▅▆▆▅▄▅▆▆█▆▆▆▅
train/learning_rate,▁▂▂▃▄▄▅▆▇▇█████████████████
+6,...


Loading tokenizer...

Preparing datasets...

Loading wikitext...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Tokenizing wikitext:   0%|          | 0/10000 [00:00<?, ? examples/s]

...


Configuring LoRA...
trainable params: 811,008 || all params: 125,250,816 || trainable%: 0.6475

🚀 Starting training...


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2285: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


Step,Training Loss,Validation Loss
500,3.219895,3.122338
1000,3.302861,3.098535
1500,3.259166,3.083264
2000,3.141431,3.077075
2500,3.174904,3.070779
3000,3.128060,3.066598
3500,3.131680,3.062268
4000,3.141486,3.059892
4500,3.127412,3.058020


Step,Training Loss,Validation Loss
500,3.219895,3.122338
1000,3.302861,3.098535
1500,3.259166,3.083264
2000,3.141431,3.077075
2500,3.174904,3.070779
3000,3.128060,3.066598
3500,3.131680,3.062268
4000,3.141486,3.059892
4500,3.127412,3.058020


KeyboardInterrupt: 

Completed checkpoint-4000

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Check checkpoint exists
import os
checkpoint_path = '/content/drive/MyDrive/ai/checkpoint-5000'
print("Files in checkpoint:", os.listdir(checkpoint_path))

# Should see: adapter_model.safetensors, adapter_config.json, etc.

ValueError: mount failed

In [ ]:
cd ..

/content/lora-slm-finetuning


In [ ]:
"""
Enhanced training script with comprehensive W&B metrics logging, resuming from a specific checkpoint.
"""
import os
import yaml
import torch
import wandb
import time
import math
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    TrainerCallback
)
from peft import LoraConfig, get_peft_model, TaskType
from data_loader import load_config, prepare_datasets, get_tokenizer
import numpy as np

class DetailedMetricsCallback(TrainerCallback):
    """Custom callback for detailed metrics logging"""

    def __init__(self):
        self.step_start_time = None
        self.ema_loss = None
        self.ema_alpha = 0.9

    def on_step_begin(self, args, state, control, **kwargs):
        self.step_start_time = time.time()

    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % args.logging_steps == 0:
            # Calculate step time
            step_time = time.time() - self.step_start_time

            # Get current loss
            current_loss = state.log_history[-1].get('loss', 0)

            # Calculate EMA loss
            if self.ema_loss is None:
                self.ema_loss = current_loss
            else:
                self.ema_loss = self.ema_alpha * self.ema_loss + (1 - self.ema_alpha) * current_loss

            # Calculate perplexity
            ppl = math.exp(current_loss) if current_loss < 100 else float('inf')

            # Calculate bits per token (bpt = loss / ln(2))
            bpt = current_loss / math.log(2)

            # Calculate throughput
            # Effective batch size = per_device_batch_size * gradient_accumulation_steps * num_gpus
            effective_batch_size = args.per_device_train_batch_size * args.gradient_accumulation_steps
            tokens_per_batch = effective_batch_size * args.max_steps  # Approximate
            throughput = tokens_per_batch / step_time if step_time > 0 else 0

            # Log additional metrics
            wandb.log({
                'train/token_loss': current_loss,
                'train/sequence_loss': current_loss,  # Same for causal LM
                'train/ppl': ppl,
                'train/bpt': bpt,
                'train/step_time': step_time,
                'train/ema_token_loss': self.ema_loss,
                'train/throughput_tokens_per_sec': throughput,
                'step': state.global_step,
            })

def train_checkpoint(
    model_config_path='configs/training_config.yaml',
    dataset_config_path='configs/datasets.yaml',
    resume_checkpoint_path="/content/checkpoint-5000"
):
    """
    Enhanced training function with comprehensive metrics,
    resuming from a specific checkpoint.
    """

    # Load configurations
    model_config = load_config(model_config_path)
    dataset_config = load_config(dataset_config_path)

    # Initialize W&B with detailed config
    run_config = {
        'model_name': model_config['model']['name'],
        'lora_r': model_config['lora']['r'],
        'lora_alpha': model_config['lora']['lora_alpha'],
        'learning_rate': model_config['training']['learning_rate'],
        'batch_size': model_config['training']['per_device_train_batch_size'],
        'grad_accum_steps': model_config['training']['gradient_accumulation_steps'],
        'epochs': model_config['training']['num_train_epochs'],
        'max_length': model_config['data']['max_length'],
        'fp16': model_config['training']['fp16'],
        'seed': model_config['training']['seed'],
        'num_datasets': len(dataset_config['datasets']),
        'resume_checkpoint': resume_checkpoint_path
    }

    wandb.init(
        project=model_config['wandb']['project'],
        entity=model_config['wandb'].get('entity'),
        tags=model_config['wandb']['tags'] + ['resumed_checkpoint'],
        config=run_config
    )

    # Log important hyperparameters
    wandb.config.update({
        'train/optimizer': 'AdamW',
        'train/precision': 'fp16' if model_config['training']['fp16'] else 'fp32',
        'train/lora_rank': model_config['lora']['r'],
        'train/grad_accum_steps': model_config['training']['gradient_accumulation_steps'],
        'train/weight_decay': model_config['training']['weight_decay'],
    })

    # Load tokenizer
    print("Loading tokenizer...")
    tokenizer = get_tokenizer(model_config['model']['name'])

    # Load datasets
    print("\nPreparing datasets...")
    train_dataset, eval_dataset = prepare_datasets(
        dataset_config,
        tokenizer,
        max_length=model_config['data']['max_length']
    )

    # Log dataset info
    wandb.log({
        'dataset/train_samples': len(train_dataset),
        'dataset/eval_samples': len(eval_dataset),
        'dataset/num_datasets': len(dataset_config['datasets']),
    })

    # Load base model
    print(f"\nLoading model: {model_config['model']['name']}")
    model = AutoModelForCausalLM.from_pretrained(
        model_config['model']['name'],
        cache_dir=model_config['model'].get('cache_dir'),
        torch_dtype=torch.float16 if model_config['training']['fp16'] else torch.float32
    )

    # Configure LoRA
    print("\nConfiguring LoRA...")
    lora_config = LoraConfig(
        r=model_config['lora']['r'],
        lora_alpha=model_config['lora']['lora_alpha'],
        lora_dropout=model_config['lora']['lora_dropout'],
        target_modules=model_config['lora']['target_modules'],
        bias=model_config['lora']['bias'],
        task_type=TaskType.CAUSAL_LM
    )

    # Apply LoRA
    model = get_peft_model(model, lora_config)
    trainable_params = model.print_trainable_parameters()

    # Training arguments
    # Create output directory for this run to avoid potential conflicts with previous run
    # OR maintain the same directory if intention is to continue filling it.
    output_dir = model_config['training']['output_dir']

    print(f"\nConfiguration overwritten: save_steps set to 500")
    print(f"Resuming from checkpoint: {resume_checkpoint_path}")

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=model_config['training']['num_train_epochs'],
        per_device_train_batch_size=model_config['training']['per_device_train_batch_size'],
        per_device_eval_batch_size=model_config['training']['per_device_eval_batch_size'],
        gradient_accumulation_steps=model_config['training']['gradient_accumulation_steps'],
        learning_rate=model_config['training']['learning_rate'],
        weight_decay=model_config['training']['weight_decay'],
        warmup_steps=model_config['training']['warmup_steps'],
        logging_steps=model_config['training']['logging_steps'],
        save_steps=500, # Hardcoded per request
        eval_steps=model_config['training']['eval_steps'],
        save_total_limit=model_config['training']['save_total_limit'],
        fp16=model_config['training']['fp16'],
        gradient_checkpointing=model_config['training']['gradient_checkpointing'],
        max_grad_norm=model_config['training']['max_grad_norm'],
        seed=model_config['training']['seed'],
        report_to="wandb",
        eval_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        logging_first_step=True,
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False
    )

    # Initialize custom callback
    metrics_callback = DetailedMetricsCallback()

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        callbacks=[metrics_callback],
    )

    # Train
    print("\n🚀 Starting training (Resuming)...")
    start_time = time.time()

    # Resume from strict checkpoint
    train_result = trainer.train(resume_from_checkpoint=resume_checkpoint_path)

    total_time = time.time() - start_time

    # Log final training metrics
    wandb.log({
        'train/total_time_hours': total_time / 3600,
        'train/samples_per_sec': len(train_dataset) * training_args.num_train_epochs / total_time,
    })

    # Save final model
    print("\n💾 Saving final model...")
    final_output_path = os.path.join(output_dir, 'final_model_resumed')
    trainer.save_model(final_output_path)

    # Final evaluation
    print("\n📊 Final evaluation...")
    eval_results = trainer.evaluate()

    # Log evaluation metrics with val/ prefix
    val_metrics = {
        'val/token_loss': eval_results['eval_loss'],
        'val/ppl': math.exp(eval_results['eval_loss']),
        'val/bpt': eval_results['eval_loss'] / math.log(2),
        'val/n_tokens': len(eval_dataset) * model_config['data']['max_length'],
        'val/n_sequences': len(eval_dataset),
    }
    wandb.log(val_metrics)

    print("\n✅ Final Results:")
    for key, value in val_metrics.items():
        print(f"  {key}: {value:.4f}")

    wandb.finish()
    print("\n✅ Training complete!")

    return trainer, eval_results

if __name__ == "__main__":
    train_checkpoint()


Loading tokenizer...

Preparing datasets...

Loading wikitext...
✓ Loaded 10000 samples from wikitext

Loading roneneldan/TinyStories...


Tokenizing roneneldan/TinyStories:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

✓ Loaded 10000 samples from roneneldan/TinyStories

Loading ag_news...
✓ Loaded 10000 samples from ag_news

Loading yelp_review_full...


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

...

Step,Training Loss,Validation Loss
5500,3.138830,3.054663
6000,3.103437,3.054175



💾 Saving final model...

📊 Final evaluation...



✅ Final Results:
  val/token_loss: 3.0542
  val/ppl: 21.2037
  val/bpt: 4.4062
  val/n_tokens: 6144000.0000
  val/n_sequences: 12000.0000


dataset/eval_samples,▁
dataset/num_datasets,▁
dataset/train_samples,▁
eval/loss,█▁▁
eval/runtime,▃█▁
eval/samples_per_second,▆▁█
eval/steps_per_second,▆▁█
step,▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇████
train/bpt,▁███████████████████████████████████████
train/ema_token_loss,▁▂▃▄▅▇▇▇████████▇▇▇███████████▇█████████
+17,...



✅ Training complete!
